# Tensorflow Core

**Knowing TensorFlow Core is valuable for the following reasons:**

    - Experimentation and debugging are both more straight forward when you can use low level TensorFlow operations directly.
    
    - It gives you a mental model of how things work internally when using the higher level APIs

### Keyterms 
    
   #### Tensor
    The central unit of data in TensorFlow
               
    A tensor consists of a set of primitive values shaped into an array of any number of dimensions.
   
   #### Rank
    Number of dimensions

   
   #### Shape
    Tuple of integers describing array's length along each dimension.
    
   #### TensorFlow Core Walkthrough
    TensorFlow Core programs as consisting of two discrete sections:
    
            1. Building a Computational graph (tf.Graph)
            
            2. Running the Computational graph (tf.Session)
   
   #### Graph
      A computational graph is a **series of TensorFlow operations** arranged into a graph. 
         
      The graph is composed of two types of objects.
              1. tf.Operation - The nodes of the graph. Operations describe calculations that consume and produce tensors.
              
              2. tf.Tensor - The edges in the graph. These represent the values that will flow through the graph. 

In [2]:
# Basic Tensorflow operation
import tensorflow as tf
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [4]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

### Feeding
     A graph can be parameterized to accept external inputs, known as placeholders.
     
     sess.run( result, feed_dict = {data : value})

In [6]:
sess = tf.Session()
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(sess.run(z, feed_dict={x: 3, y: 4.5}))


7.5


# Datasets
    
   #### Placeholders work for simple experiments, but tf.data are the preferred method of streaming data into a model.
   

In [14]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


# Layers

   - A trainable model must modify the values in the graph to get new outputs with the same input. tf.layers are the preferred way to add trainable parameters to a graph.
    
   - a densely-connected layer performs a weighted sum across all inputs for each output and applies an optional activation function.


## Creating Layers
    

In [20]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)(x)

print(linear_model)

#Initializing Layers
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(linear_model,feed_dict={x:[[1,2,3]]}))

Tensor("dense_5/BiasAdd:0", shape=(?, 1), dtype=float32)
[[0.57237625]]


In [32]:
import numpy as np
x = tf.constant(np.arange(1,11).reshape((10,1)), dtype=tf.float32)
y_true = tf.constant(np.square(np.arange(1,11)).reshape((10,1)), dtype=tf.float32)

Model = tf.layers.Dense(units=1)
y_pred = Model(x)

In [34]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[0.64026606]
 [1.2805321 ]
 [1.9207982 ]
 [2.5610642 ]
 [3.2013302 ]
 [3.8415964 ]
 [4.4818625 ]
 [5.1221285 ]
 [5.7623944 ]
 [6.4026604 ]]


## Loss

    To optimize a model, you first need to define the loss. We'll use the mean square error, a standard loss for regression problems.
   

In [35]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

2161.7217


## Training

   - TensorFlow provides optimizers implementing standard optimization algorithms.
    
   - These are implemented as sub-classes of **tf.train.Optimizer**. 
        
   - They incrementally change each variable in order to minimize the loss.

In [36]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [43]:
for i in range(10):
  _, loss_value = sess.run((train, loss))
  

In [44]:
print(sess.run(y_pred))


[[-1.0999800e+01]
 [ 1.6593933e-04]
 [ 1.1000132e+01]
 [ 2.2000097e+01]
 [ 3.3000061e+01]
 [ 4.4000031e+01]
 [ 5.4999992e+01]
 [ 6.5999962e+01]
 [ 7.6999931e+01]
 [ 8.7999893e+01]]
